# Classiq Challenge Solution

Submitted by team Quantotto

In [1]:
from classiq import *
import numpy as np
import math

In [13]:
def norm(a):
    b = np.array(a)
    return np.sqrt(np.einsum('i,i', b, b))

def normalize(a):
    return list(np.array(a) / norm(a))

def sqrt_sum(a):
    return np.sqrt(np.sum(np.array(a)))

In [14]:
def calculate_Cs(x0, t, k):
    x0_norm = norm(x0)
    arr = [x0_norm * (t ** i) / math.factorial(i) for i in range(k+1)]
    return arr

In [56]:
def calculate_V(Cs):
    return [[1, 0], [0, -1]]

def calculate_Vs1(Cs, k):
    assert k == 1
    C = sqrt_sum(Cs)
    return [[np.sqrt(Cs[0]) / C, np.sqrt(Cs[1]) / C], [-np.sqrt(Cs[1]) / C, np.sqrt(Cs[0]) / C]]

In [57]:
@qfunc
def prep_qubits(Vs1: CArray[CArray[CReal]], normalized_x0: CArray[CReal], approximator: QBit, phi: QBit):
    unitary(Vs1, approximator)
    inplace_prepare_amplitudes(normalized_x0, bound=0.01, target=phi)

@qfunc
def encode(V: CArray[CReal], Vs1: CArray[CArray[CReal]], normalized_x0: CArray[CReal], ancilla: Output[QBit], approximator: Output[QBit], phi: QBit):
    allocate(1, ancilla)
    allocate(1, approximator)
    unitary(V, ancilla)
    control(ctrl=ancilla==0, stmt_block=lambda: prep_qubits(Vs1, normalized_x0, approximator, phi))

@qfunc
def approximate(A: CArray[CArray[CReal]], approximator: QBit, phi: QBit):
    control(ctrl=approximator==0, stmt_block=lambda: apply_to_all(IDENTITY, phi))
    control(ctrl=approximator==1, stmt_block=lambda: unitary(A, phi))

In [60]:
# inputs for y'' + y = 0
x0 = [1.0, 1.0]                    # Boundary conditions
A = [[0, 1], [-1, 0]]              # Linear equations (unitary)
t = np.pi / 8                      # evolution time (calculating x(t))
k = 1                              # order of approximation

Cs = calculate_Cs(x0, t, k)
V = calculate_V(Cs)
Vs1 = calculate_Vs1(Cs, k)
normalized_x0 = normalize(x0)

@qfunc
def main(phi: Output[QBit]):
    ancilla = QBit("ancilla")
    approximator = QBit("approximator")
    allocate(1, phi)
    within_apply(
        within=lambda: encode(V, Vs1, normalized_x0, ancilla, approximator, phi),
        apply=lambda: approximate(A, approximator, phi)
    )

In [62]:
qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

In [68]:
write_qmod(create_model(main), "diffeq")

In [67]:
np.pi / np.arcsin(np.sqrt(0.282))

5.6117547256097104